In [2]:
import os, errno
from os import path
import requests
from bs4 import BeautifulSoup as bs
import regex as re
import copy
import pandas as pd
from urlparse import urlparse
from datetime import datetime


In [168]:
mol_diag_url = 'https://www.genomeweb.com/molecular-diagnostics'
r = requests.get(mol_diag_url)

In [169]:
r.status_code

200

In [170]:
r.content;

In [171]:
soup = bs(r.content, "lxml")

In [175]:
print(soup.prettify());

<!DOCTYPE html>
<html dir="ltr" lang="en" xmlns:article="http://ogp.me/ns/article#" xmlns:book="http://ogp.me/ns/book#" xmlns:content="http://purl.org/rss/1.0/modules/content/" xmlns:dc="http://purl.org/dc/terms/" xmlns:foaf="http://xmlns.com/foaf/0.1/" xmlns:og="http://ogp.me/ns#" xmlns:product="http://ogp.me/ns/product#" xmlns:profile="http://ogp.me/ns/profile#" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#" xmlns:schema="http://schema.org/" xmlns:sioc="http://rdfs.org/sioc/ns#" xmlns:sioct="http://rdfs.org/sioc/types#" xmlns:skos="http://www.w3.org/2004/02/skos/core#" xmlns:video="http://ogp.me/ns/video#" xmlns:xsd="http://www.w3.org/2001/XMLSchema#">
 <head profile="http://www.w3.org/1999/xhtml/vocab">
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <script type="text/javascript">
   window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var o=n[t]={exports:{}};e[t]

In [191]:
div = soup.find("div", {"class": "clearfix fieldlayout-region-top"})
#div = soup.find("div", {"class": "node-title fieldlayout node-field-title"})  
print div

<div class="clearfix fieldlayout-region-top"><h3 class=" node-title fieldlayout node-field-title"><a href="/business-news/startup-spindiag-raises-16m-seed-funding">Startup SpinDiag Raises €1.6M in Seed Funding</a></h3></div>


In [192]:
div.find('a')['href']

'/business-news/startup-spindiag-raises-16m-seed-funding'

In [16]:
print soup.title

<title>Molecular Diagnostics News | GenomeWeb</title>


In [3]:
def get_pages(base_url='https://www.genomeweb.com',
            subsection = '/molecular-diagnostics'):
    ''' This function takes a particular url and subsection from genomeweb as input 
    and returns a list with the complete set of pages following the next button '''    
    
    # Define url
    url = base_url+subsection
    page_list = []
    page_list.append(url)
    r = requests.get(url)
    soup = bs(r.content)
    
    #This checks if we start on first page - enumerated differently from the rest in genomeweb
    if soup.find("div", {"class": "more-link"}):
        
        # finds division for class more link
        div = soup.find("div", {"class": "more-link"})  
        
        #extract the hyperlink from the division 
        elm = div.find('a')['href']
        
        # makes a link for next page based on base url and hiperlink 
        next_page_link = base_url + elm
        page_list.append(next_page_link)
        
        # extract data from second page in genomeweb - first enumerated as the rest 
        r = requests.get(next_page_link)
        soup = bs(r.content)
        page_list.append(next_page_link)
    else:    
        next_page_link = url
    
    # This loop will start with the second page or higher and immediatly move to next one
    # until last page has been listed
    while soup.find('a', {'title': 'Go to next page'}) :
        print next_page_link
        
        # extract hyperlink for next page
        elm = soup.find('a', {'title': 'Go to next page'})['href']
        next_page_link = base_url + elm
        r=r = requests.get(next_page_link)
        soup = bs(r.content)
        page_list.append(next_page_link)
    
    return page_list

In [4]:
page_list = get_pages('https://www.genomeweb.com','/channel/33?page=270')

/Users/ale/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /Users/ale/anaconda2/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


https://www.genomeweb.com/channel/33?page=270
https://www.genomeweb.com/channel/33?page=271
https://www.genomeweb.com/channel/33?page=272
https://www.genomeweb.com/channel/33?page=273
https://www.genomeweb.com/channel/33?page=274


In [174]:
def scrape_page(page_list):
    
    # regular expression to find text between and including > <: '>text<'
    regex = r'\>(.*?)\<'
   
    # regular expression to find text between words title and typeof --> not included
    regex2 = r'(?<=title).*?(?=" typeof)'#        
    
    # Make empty data frame where information will be updated
    columns = ['Title','Hyperlink', 'Info', 'Date']
    df = pd.DataFrame(columns = columns)
    
    # loop through pages in page list
    for page in page_list:
        #print page
        
    # extract base url for information
        parsed_uri = urlparse(page)
        base_url = '{uri.scheme}://{uri.netloc}'.format(uri=parsed_uri)
        
        
    # open page and beautify
        html = requests.get(page)
        soup = bs(html.content, "lxml")
        
    # divison with all pertinent information
        #div = soup.find_all("div", {"class": "view-content"})[0]
        div = soup.find("div", {"class": "view-content"})
        #print div
        #a = soup.find_all("a", class_="vew-content")
        a = div.find_all('a') # find anchor containing hyperlink
        print a
        p = div.find_all('p') # find paragraph with information
        d = soup.find_all('span',{'class':'date-display-single'}) # find date of article publication
        
    # Some pages have repeated hyperlinks - we defined hiperlink_old and dummy variable to avoid repeat rows
        hiperlink_old = ''
    
    # loop through all links, date and extracted paragraph together
        for link,doc,info in zip(a,d,p):
 
            #print link
            hiperlink = str(base_url+link['href']) # extract hyperlink from anchor link
            
    # because of inconsistencies on webpages title can be extracted two ways
    # match1 and macth2 cover both options
            match1 = re.search(regex, str(link))
            match2 = re.search(regex2,str(link))
            
    # when match2 exists, title for macth1 is innacurate 
            if match2:
                title = match2[0][2:]+'.'
            else:
                title = match1[0][1:-1]   
            
                
    # extract date
            date = doc['content']
            date = date[:10]
            
            info = str(info)[3:-4]
            
    # if hyperlink is not repeated add all new information to dataframe
            if hiperlink != hiperlink_old:
                df = df.append({'Hyperlink': hiperlink, 'Title':title,'Date': date, 'Info':info}, ignore_index=True) 
            hiperlink_old = hiperlink
  
    
    print 'information extrcated from: ',base_url
    return df
    


#scrape_page(['https://www.genomeweb.com/channel/33?page=273'])
#df = scrape_page(['https://www.genomeweb.com/channel/33?page=1'])

In [33]:
page_list2 =get_pages()

https://www.genomeweb.com/channel/33?page=1
https://www.genomeweb.com/channel/33?page=2
https://www.genomeweb.com/channel/33?page=3
https://www.genomeweb.com/channel/33?page=4
https://www.genomeweb.com/channel/33?page=5
https://www.genomeweb.com/channel/33?page=6
https://www.genomeweb.com/channel/33?page=7
https://www.genomeweb.com/channel/33?page=8
https://www.genomeweb.com/channel/33?page=9
https://www.genomeweb.com/channel/33?page=10
https://www.genomeweb.com/channel/33?page=11
https://www.genomeweb.com/channel/33?page=12
https://www.genomeweb.com/channel/33?page=13
https://www.genomeweb.com/channel/33?page=14
https://www.genomeweb.com/channel/33?page=15
https://www.genomeweb.com/channel/33?page=16
https://www.genomeweb.com/channel/33?page=17
https://www.genomeweb.com/channel/33?page=18
https://www.genomeweb.com/channel/33?page=19
https://www.genomeweb.com/channel/33?page=20
https://www.genomeweb.com/channel/33?page=21
https://www.genomeweb.com/channel/33?page=22
https://www.genomew

https://www.genomeweb.com/channel/33?page=182
https://www.genomeweb.com/channel/33?page=183
https://www.genomeweb.com/channel/33?page=184
https://www.genomeweb.com/channel/33?page=185
https://www.genomeweb.com/channel/33?page=186
https://www.genomeweb.com/channel/33?page=187
https://www.genomeweb.com/channel/33?page=188
https://www.genomeweb.com/channel/33?page=189
https://www.genomeweb.com/channel/33?page=190
https://www.genomeweb.com/channel/33?page=191
https://www.genomeweb.com/channel/33?page=192
https://www.genomeweb.com/channel/33?page=193
https://www.genomeweb.com/channel/33?page=194
https://www.genomeweb.com/channel/33?page=195
https://www.genomeweb.com/channel/33?page=196
https://www.genomeweb.com/channel/33?page=197
https://www.genomeweb.com/channel/33?page=198
https://www.genomeweb.com/channel/33?page=199
https://www.genomeweb.com/channel/33?page=200
https://www.genomeweb.com/channel/33?page=201
https://www.genomeweb.com/channel/33?page=202
https://www.genomeweb.com/channel/

In [175]:
scrape_page(page_list2[:1])

[<a href="/molecular-diagnostics/new-england-biolabs-ttp-partner-provide-mdx-development-services">New England Biolabs, TTP Partner to Provide MDx Development Services</a>, <a href="/molecular-diagnostics/oneome-inks-deal-offer-pgx-test-canada">OneOme Inks Deal to Offer PGx Test in Canada</a>, <a href="/sequencing/panel-sequencing-leads-pathogenic-mutations-autoinflammatory-cases">Panel Sequencing Leads to Pathogenic Mutations in Autoinflammatory Cases</a>]
information extrcated from:  https://www.genomeweb.com


,Title,Hyperlink,Info,Date
0,"New England Biolabs, TTP Partner to Provide MD...",https://www.genomeweb.com/molecular-diagnostic...,"dir=""ltr"">The services will couple NEB's reage...",2017-07-27
1,OneOme Inks Deal to Offer PGx Test in Canada,https://www.genomeweb.com/molecular-diagnostic...,The RightMed test covers 22 genes in which var...,2017-07-27
2,Panel Sequencing Leads to Pathogenic Mutations...,https://www.genomeweb.com/sequencing/panel-seq...,"With new gene sequencing panels, investigators...",2017-07-27


In [107]:
page_list2


['https://www.genomeweb.com/molecular-diagnostics',
 'https://www.genomeweb.com/channel/33?page=1',
 'https://www.genomeweb.com/channel/33?page=1',
 'https://www.genomeweb.com/channel/33?page=2',
 'https://www.genomeweb.com/channel/33?page=3',
 'https://www.genomeweb.com/channel/33?page=4',
 'https://www.genomeweb.com/channel/33?page=5',
 'https://www.genomeweb.com/channel/33?page=6',
 'https://www.genomeweb.com/channel/33?page=7',
 'https://www.genomeweb.com/channel/33?page=8',
 'https://www.genomeweb.com/channel/33?page=9',
 'https://www.genomeweb.com/channel/33?page=10',
 'https://www.genomeweb.com/channel/33?page=11',
 'https://www.genomeweb.com/channel/33?page=12',
 'https://www.genomeweb.com/channel/33?page=13',
 'https://www.genomeweb.com/channel/33?page=14',
 'https://www.genomeweb.com/channel/33?page=15',
 'https://www.genomeweb.com/channel/33?page=16',
 'https://www.genomeweb.com/channel/33?page=17',
 'https://www.genomeweb.com/channel/33?page=18',
 'https://www.genomeweb.com